In [72]:
from math import floor
from typing import Tuple
from Crypto.Util.number import getPrime


def mod_inverse(a, m):
    gcd, x, _ = egcd(a, m)
    assert gcd == 1, f"the modular multiplicative inverse does not exist for a: {a}, m: {m}"
    return x


def egcd(a, b):
    """
    Algorithm outlined at https://en.m.wikipedia.org/wiki/Extended_Euclidean_algorithm

    NOTE: must satisfy |a| > |b|
    """
    r_prev = a
    r = b
    s_prev = 1
    s = 0
    t_prev = 0
    t = 1
    while True:
        q = floor(r_prev / r)
        r_next = r_prev - q * r
        s_next = s_prev - q * s
        t_next = t_prev - q * t
        if r_next == 0:
            return r, s, t
        r_prev = r
        r = r_next
        s_prev = s
        s = s_next
        t_prev = t
        t = t_next


def generate_keys(prime_size: int = 512):
    e = 65537
    while True:
        p = getPrime(prime_size)
        q = getPrime(prime_size)
        totient = (p - 1) * (q - 1)
        if egcd(e, totient)[0] == 1:  # ensure e is coprime with totient
            break
    
    n = p * q
    d = mod_inverse(e, totient)  # modular multiplicative inverse of e mod totient
    return (n, e), (n, d)


def encrypt(m: str, pubkey: Tuple[int, int]) -> int:
    m = int.from_bytes(m.encode())
    n, e = pubkey
    return pow(m, e, n)


def decrypt(c: int, privkey: Tuple[int, int]) -> str:
    n, d = privkey
    m = pow(c, d, n)
    return m.to_bytes((m.bit_length() + 7) // 8).decode()


message = "Hello, world!"
pubkey, privkey = generate_keys()
ciphertext = encrypt(message, pubkey)
recovered_message = decrypt(ciphertext, privkey)
recovered_message

1024


'Hello, world!'